In [6]:
from show_result import show_result
import os
for f in os.listdir("results/tests_1014")[::-1]:
    print("\033[32m> {}\033[0m".format(f))
    show_result(f"results/tests_1014/{f}")
    print()

> exp_7_lstm_80_mask

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[     lstm] -	-	-	-	-	-	-	-	-	-	-	-

> exp_6_mpnn_lstm_80_mask

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[mpnn_lstm] -	-	-	-	-	-	-	-	-	-	-	-

> exp_5_lstm

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[     lstm] 5.333	-	-	-	5.777	-	-	-	5.335	-	-	-

> exp_4_missing_nodes_80_mask

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] -	-	-	-	-	-	-	-	-	-	-	-

> exp_3_missing_nodes_no_mask

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 5.539	1.46	7.476	24.212	5.62	1.676	7.992	33.099	6.161	1.867	11.784	28.034

> exp_2_dynst_gt_only

[err_test] England,France,Italy,Spain

7->1 (w7s2) | 7->1 (w7s6) | 7->1 (w7s13)
[    dynst] 5.436	1.486	7.437	31.749	4.894	1.463	8.08	39.035	6.637	1.879	11.299	28.035

> exp_1_baseline_mpnn_lstm

[err_test

In [2]:
raise NotImplementedError
import os

def delete_pth_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pth'):
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

# 使用示例
delete_pth_files('results/tests_old')


NotImplementedError: 